In [1]:
import io
import csv

import psycopg2
from getpass import getpass

from sqlalchemy import create_engine, text
from tqdm.auto import tqdm

import pandas as pd
import numpy as np

In [2]:
db_params = dict(user=f"{input('User: ')}@look-inna-book-postgres-hasura-pg-server",
                 password=getpass('Password: '),
                 host="look-inna-book-postgres-hasura-pg-server.postgres.database.azure.com",
                 port="5432",
                 database="look-inna-book"
                )

User: jacobgdt
Password: ········


In [3]:
conn_str = 'postgresql://{user}:{password}@{host}:{port}/{database}'
engine = create_engine(conn_str.format(**db_params))

engine.table_names()

['final."Book"',
 'PaymentInfo',
 'ShippingAddress',
 'Order',
 'Store',
 'authorbook',
 'purchaseditem',
 'paymentmethod',
 'author',
 'book',
 'shipment',
 'cardholder',
 'residentialaddress',
 'customer',
 'household',
 'phonenumber',
 'purchase',
 'Book',
 'Offer',
 'publisher']

## Data Generation Status

Currently automating creation of:

* ✅ author
* ✅ book
* ✅ cardholder 
* ✅ customer 
* ✅ household 
* ✅ paymentmethod 
* ❌ phonenumber
* ❌ publisher
* ✅ purchase 
* ✅ purchaseditem 
* ❌ residentialaddress
* ❌ shipment
* ✅ writtenby 


## Create Schema

In [4]:
# This doesn't work for whatever reason.
# Can just run in pgAdmin or Hasura.

with open('../SQL/DDL.sql') as f:
    schema = f.read()
    
engine.execute(text(schema))

ProgrammingError: (psycopg2.errors.SyntaxError) syntax error at or near "foreign"
LINE 76:   foreign key (cardNumber) references PaymentMethod,
           ^

[SQL: -- set schema
CREATE SCHEMA IF NOT EXISTS final;
SET search_path TO final; 

CREATE TABLE IF NOT EXISTS ResidentialAddress(
  residenceID serial primary key, 
  firstname text,
  lastname text,
  street text,
  city text,
  province text,
  country text,
  postalCode text
);

CREATE TABLE IF NOT EXISTS Publisher( /* Finished Assumed publisher only has one address..*/
  publisherID serial primary key,
  publisherName text,
  bankingInformation text,
  residenceID integer,

  foreign key (residenceID) references ResidentialAddress
);

CREATE TABLE IF NOT EXISTS Book (
  asin char(10) primary key, -- ASIN are guaranteed to be size 10
  publisherID integer,
  title text,
  coverURL text DEFAULT NULL,
  category text,
  subCategory text,
  pages integer,
  price numeric(6, 2),
  inventory integer,
  percentageTaken numeric(3,2) D

## Inserting Book Data

In [5]:
df = pd.read_csv('./amazon_books.csv') \
        .drop(columns=['filename', 'category_id']) \
        .rename(columns={
            'ASIN': 'asin',
            'image_url': 'coverURL'
        })

random_field = lambda low, high: np.random.randint(low, high, (len(df),))
df = df.assign(
    asin=lambda df: df.asin.apply(lambda asin: asin[:10]),
    publisherID=np.nan,
    pages=random_field(100, 1000),
    price=(np.random.rand() * random_field(2,50)).round(2),
    inventory=random_field(12, 100)
)

dump = df.drop(columns="author")
dump.head()

df.head()

,asin,coverURL,title,author,category,publisherID,pages,price,inventory
0,761183272,http://ecx.images-amazon.com/images/I/61Y5cOdH...,Mom's Family Wall Calendar 2016,Sandra Boynton,Calendars,NaN,298,16.57,44
1,1623439671,http://ecx.images-amazon.com/images/I/61t-hrSw...,Doug the Pug 2016 Wall Calendar,Doug the Pug,Calendars,NaN,964,1.69,38
2,B00O80WC6I,http://ecx.images-amazon.com/images/I/41X-KQqs...,"Moleskine 2016 Weekly Notebook, 12M, Large, Bl...",Moleskine,Calendars,NaN,564,3.04,54
3,761182187,http://ecx.images-amazon.com/images/I/61j-4gxJ...,365 Cats Color Page-A-Day Calendar 2016,Workman Publishing,Calendars,NaN,300,13.52,44
4,1578052084,http://ecx.images-amazon.com/images/I/51Ry4Tsq...,Sierra Club Engagement Calendar 2016,Sierra Club,Calendars,NaN,677,3.04,37


In [6]:
# https://stackoverflow.com/a/55495065/6766123
def psql_insert_copy(table, conn, keys, data_iter):
    # gets a DBAPI connection that can provide a cursor
    data_iter = tqdm(data_iter)
    dbapi_conn = conn.connection
    
    with dbapi_conn.cursor() as cur:
        s_buf = io.StringIO()
        writer = csv.writer(s_buf)
        writer.writerows(data_iter)
        s_buf.seek(0)

        columns = ', '.join('"{}"'.format(k) for k in keys)
        if table.schema:
            table_name = '{}.{}'.format(table.schema, table.name)
        else:
            table_name = table.name

        sql = 'COPY {} ({}) FROM STDIN WITH CSV'.format(table_name, columns)
        cur.copy_expert(sql=sql, file=s_buf)

In [7]:
dump.columns = [c.lower() for c in dump.columns]
dump.to_sql('book', engine, schema="final", if_exists="append", index=False, method=psql_insert_copy)

## Inserting Author Data

In [12]:
# delete all rows; can't use replace because of dependencies
engine.execute('truncate final."author" cascade;') 

seq_key = engine.execute("""SELECT pg_get_serial_sequence('final."author"', 'authorid');""").fetchall()[0][0]
reset_pk = "ALTER SEQUENCE {} RESTART WITH 1".format(seq_key)
engine.execute(reset_pk)

In [13]:
author_dump = df[['author', 'asin']].rename(columns={'author': 'fullname'}).dropna()
author_dump[['fullname']] \
    .drop_duplicates(subset='fullname') \
    .to_sql('author', engine, schema="final", if_exists="append", index=False, method=psql_insert_copy)

In [14]:
engine.execute('SELECT COUNT(*) FROM final."author"').fetchall()

[(117374,)]

In [15]:
query = engine.execute('SELECT * FROM final."author"')
author_ids = pd.DataFrame(query.fetchall(), columns=['authorid', 'fullname'])

author_ids.head()

,authorid,fullname
0,1,Sandra Boynton
1,2,Doug the Pug
2,3,Moleskine
3,4,Workman Publishing
4,5,Sierra Club


In [16]:
writtenby = pd.merge(author_dump, author_ids, on='fullname')

print(writtenby.shape)
writtenby.head()

(193159, 3)


,fullname,asin,authorid
0,Sandra Boynton,761183272,1
1,Sandra Boynton,761177817,1
2,Sandra Boynton,761185631,1
3,Sandra Boynton,761137998,1
4,Sandra Boynton,894801996,1


In [18]:
writtenby.drop(columns='fullname') \
        .to_sql('writtenby', engine, schema="final", if_exists="append", index=False, method=psql_insert_copy)

## Inserting Users

In [ ]:
def random_uuids(n):
    return (str(uuid.uuid4()) for _ in range(n))

In [ ]:
import uuid

NUM_USERS = 1_000

users = pd.DataFrame([{'userid': uuid} for uuid in random_uuids(NUM_USERS)])
users.head()

In [ ]:
users.to_sql('customer', engine, schema="final", if_exists="append", index=False, method=psql_insert_copy)

## Inserting Credit Cards

In [ ]:
from faker import Faker

fake = Faker()
Faker.seed(0)

def random_credit_cards(n):
    for _ in range(n):
        yield {
            'cardnumber': fake.credit_card_number(card_type='visa'),
            'cvv': fake.credit_card_security_code(card_type='visa'),
            'expirationdate': fake.credit_card_expire(end='+5y', date_format='20%y-%m-01')
        }

In [ ]:
NUM_CARDS = int(NUM_USERS*1.75)

random_cardholders = users.sample(NUM_CARDS, replace=True).values.squeeze()
random_cc_iter = zip(random_cardholders, random_credit_cards(NUM_CARDS))

cc_users = pd.DataFrame([{'userid': uid, **cc} for (uid, cc) in random_cc_iter])

print(cc_users.shape)
cc_users.head()

In [ ]:
payment_method = cc_users.drop(columns='userid')
payment_method.to_sql('paymentmethod', engine, schema="final", if_exists="append", index=False, method=psql_insert_copy)

In [ ]:
cardholders = cc_users[['userid', 'cardnumber']]
cardholders.to_sql('cardholder', engine, schema="final", if_exists="append", index=False, method=psql_insert_copy)

## Simulating Purchases

In [ ]:
from datetime import datetime

# https://stackoverflow.com/a/57722873/6766123

def random_dates(first_date, second_date, n):
    d1 = datetime.strptime(first_date, "%m/%d/%Y %I:%M %p")
    d2 = datetime.strptime(second_date, "%m/%d/%Y %I:%M %p")
    
    first_timestamp = int(d1.timestamp())
    second_timestamp = int(d2.timestamp())
    random_timestamps = np.random.randint(first_timestamp, second_timestamp, (n,))
    
    return pd.Series(random_timestamps).apply(datetime.fromtimestamp)

In [ ]:
NUM_PURCHASES = np.random.randint(2000, 3000)
NUM_SAMPLED_USERS = np.random.randint(NUM_USERS//3, NUM_USERS//3*2)

purchases = cardholders.sample(NUM_SAMPLED_USERS) \
                .sample(NUM_PURCHASES, replace=True) \
                .assign(
                    orderid=list(random_uuids(NUM_PURCHASES)),
                    ordertime=random_dates("1/1/2018 1:30 PM", "4/4/2020 4:50 AM", NUM_PURCHASES)
                )

assert purchases.userid.isin(cardholders.userid).all()

print(purchases.shape)
purchases.head()

In [ ]:
# https://stackoverflow.com/a/50425683/6766123
def softmax(x, axis=None):
    x = x - x.max(axis=axis, keepdims=True)
    y = np.exp(x)
    return y / y.sum(axis=axis, keepdims=True)

MAX_QTY = 4
PURCHASE_QUANTITIES = list(range(1,MAX_QTY+1))
QTY_WEIGHTS = softmax(-2*np.array(PURCHASE_QUANTITIES))
QTY_WEIGHTS

In [ ]:
NUM_PURCHASED_ITEMS = np.random.randint(8000, 12_000)
purchased_items = df[['asin']].sample(NUM_PURCHASED_ITEMS, replace=True) \
                            .assign(
                                orderid=purchases.orderid.sample(NUM_PURCHASED_ITEMS, replace=True).values,
                                quantity=pd.Series(PURCHASE_QUANTITIES) \
                                            .sample(NUM_PURCHASED_ITEMS, 
                                                    replace=True, 
                                                    weights=QTY_WEIGHTS) \
                                            .values
                            )

print(purchased_items.shape)
purchased_items.head()

In [ ]:
purchased_items.orderid.value_counts()

In [ ]:
purchased_items.quantity.value_counts()

In [ ]:
purchases.to_sql('purchase', engine, schema="final", if_exists="append", index=False, method=psql_insert_copy)

In [ ]:
purchased_items.to_sql('purchaseditem', 
                       engine, 
                       schema="final", 
                       if_exists="append", 
                       index=False, 
                       method=psql_insert_copy)